# Protein Classification for Kunitz domain with HMM
Pfam family entry: [Kunitz_BPTI](https://pfam.xfam.org/family/Kunitz_BPTI) (PF00014)

This doc lives in:

In [1]:
%%bash
pwd

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/docs


## Notes for this prj from LB1/prj_blast_classification/

### blastclust
blastclust -i ids_kunitz.fasta -o ids_kunits.clust -L 0.8 -S 90

clusters together the sequences that share 90% sequence identity for 0.8 sequence length(?). The output is sorted such that in the first line there is the cluster with the greatest number of sequences

This will be very useful for checking and removing redundancy when considering queries in PDB

### HMM classificator framework
* fetch pdb for kunitz and non-kunitz
* check for redundancy and filter
* train hmm with training set
* optimize threshold
* test hmm with different set (do some ten-fold cross-validation)
* do further tests that will be discussed later

### Fetch pdb files
* Use RCSB PDB advanced search; note that you can filter for mutations, co-crystallization...
* Use pdbefold to have a list of structures given a selected one

### Report sections
* Datasets
* Model Building
* Optimization
* Performance

### Misc
* Consider Python "set" function to perform operations on sets; intersection, union etc. e.g. useful to compare lists of identifiers
* This might be useful for finding deprecated software packages: http://lipid.biocomp.unibo.it/emidio/tmp/

## Manual, boring workflow

### pdb search

Filters:
* pfam domain 
* wildtype True
* resolution < 3.5 A
* sequence length, not to have big gaps in alignments: 50 - 90

**params**

Pfam Accession Number PF00014 and Resolution is between 0.0 and 3.5 and Sequence Length is between 50 and 90 and Percent Sequence Alignment Search : PDB can contain Expression Tag sequence = Yes , 

From the advanced search --> get results of the query --> select custom tabular report with customizable info
5PTI seems to be a good candidate for being a representative of the struture
* no other proteins linked --> active conformation
* good resolution
* has ligands, should not be a problem

Prof pbtained note: 163 unique id_chain

### pdbefold search
Submit 5PTI search

Note: 569 unique id_chain

### match the search from pdb with the search from pdbefold
Take the intersection

Side note: to implement queue management of jobs [Sun Grid Engine](https://en.wikipedia.org/wiki/Oracle_Grid_Engine)

## Handmade files
I downloaded the pdb search results and the pdbefold search results

In [2]:
%%bash
cd ../dataset/
pwd
ls -lh

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
total 0
lrwxrwxrwx 1 alessandro alessandro 35 apr  3 11:40 pdbefold_5pti_A_summary -> ../handmade/pdbefold_5pti_A_summary
lrwxrwxrwx 1 alessandro alessandro 43 apr  3 12:16 rcsbpdb_wt_PF00014_len50_90.csv -> ../handmade/rcsbpdb_wt_PF00014_len50_90.csv
lrwxrwxrwx 1 alessandro alessandro 23 apr  3 09:25 Snakefile -> ../snakefiles/Snakefile


### Parsing handmade files
#### rcsbpdb_wt_PF00014_len50_90.txt.gz
.META:

1 ID_CHAIN

2 Resolution

3 Taxonomy

4 Sequence

5 Chain Length

#### pdbefold_5pti_A_summary.txt.gz
.META:
1 ID_CHAIN

2 Q-score

3 P-score

4 Z-score

5 RMSD

In [3]:
%%bash
cd ../dataset/
pwd
snakemake -p rcsbpdb_wt_PF00014_len50_90.txt.gz
snakemake -p pdbefold_5pti_A_summary.txt.gz
zcat rcsbpdb_wt_PF00014_len50_90.txt.gz | head
zcat pdbefold_5pti_A_summary.txt.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
1AAP_A	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58
1AAP_B	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58
1B0C_A	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_B	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_C	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_D	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_E	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_A	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_B	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_C	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
5PTI_A	1	16.18	11.93	0.000
9PTI_A	0.999	14.33	11.2

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	parse_rcsbpdb_table
	1

rule parse_rcsbpdb_table:
    input: rcsbpdb_wt_PF00014_len50_90.csv
    output: rcsbpdb_wt_PF00014_len50_90.txt.gz
    jobid: 0

cat rcsbpdb_wt_PF00014_len50_90.csv | sed 's/,/	/g' | sed 's/"//g' | sed 's/ /_/g' | sed '1d' | sed 's/	/_/' | gzip > rcsbpdb_wt_PF00014_len50_90.txt.gz
Finished job 0.
1 of 1 steps (100%) done
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	parse_pdbefold_summary
	1

rule parse_pdbefold_summary:
    input: pdbefold_5pti_A_summary
    output: pdbefold_5pti_A_summary.txt.gz
    jobid: 0

cat pdbefold_5pti_A_summary | sed '1,5d' | tr -s " " "\t" | cut -f3- | awk '{print toupper($17)"	"$1"	"$2"	"$3"	"$4}' | sed 's/:/_/' | gzip > pdbefold_5pti_A_summary.txt.gz
Finished job 0.
1 of 1 steps (100%) done


## Join the tables on the intersection of identifiers

In [4]:
%%bash
cd ../dataset/
pwd
snakemake -p intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz
zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
1AAP_A	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58	0.8785	8.378	8.472	0.944
1AAP_B	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58	0.8667	7.554	8.024	0.819
1B0C_A	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9454	10.26	9.419	0.437
1B0C_B	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9463	10.26	9.419	0.428
1B0C_C	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9457	10.26	9.419	0.434
1B0C_D	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9447	10.23	9.402	0.446
1B0C_E	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9453	10.26	9.419	0.439
1BHC_A	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	0.9484	11.34	9.924	0.403
1BHC_B	2.70	Bos_taur

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	intersect_with_all_data
	1

rule intersect_with_all_data:
    input: rcsbpdb_wt_PF00014_len50_90.txt.gz, pdbefold_5pti_A_summary.txt.gz
    output: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz
    jobid: 0
    wildcards: t1=rcsbpdb_wt_PF00014_len50_90, t2=pdbefold_5pti_A_summary

join -j 1 <(zcat rcsbpdb_wt_PF00014_len50_90.txt.gz | sort) <(zcat pdbefold_5pti_A_summary.txt.gz | fgrep -f <(zcat rcsbpdb_wt_PF00014_len50_90.txt.gz | cut -f1) | sort ) | sed 's/ /	/g' | gzip > intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz
Finished job 0.
1 of 1 steps (100%) done


## Download seqres from wwPDB
wget this: http://lipid.biocomp.unibo.it/emidio/tmp/pdb_seqres.txt

Not really needed since the sequence are already stored in rcsbpdb_wt_PF00014_len50_90.csv

## run blastclust on intersection set 
Prof obtained 25 clusters using -S 99 -L 0.95; parameters are flexible, but you want at least 20 clusters

--> Sort the identifiers from the clusters by resolution (or other custom parameter, e.g the length): extract the representative one for each cluster

Do do so:

First, generate fasta file from the intersection table:

In [5]:
%%bash
cd ../dataset/
pwd
snakemake -p intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta
head intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
>1AAP_A
VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA
>1AAP_B
VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA
>1B0C_A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1B0C_B
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1B0C_C
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	intersection2fasta
	1

rule intersection2fasta:
    input: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz
    output: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta
    jobid: 0
    wildcards: t1=rcsbpdb_wt_PF00014_len50_90, t2=pdbefold_5pti_A_summary

zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz | awk '{print ">"$1"\n"$4}' > intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta
Finished job 0.
1 of 1 steps (100%) done


Then, compute the clusters of these sequences using blastclust:

In [6]:
%%bash
cd ../dataset/
pwd
snakemake -p intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz
zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
Apr 4, 2019 12:06 PM Start clustering of 162 queries
1B0C_A 1B0C_B 1B0C_C 1B0C_D 1B0C_E 1BHC_A 1BHC_B 1BHC_C 1BHC_D 1BHC_E 1BHC_F 1BHC_G 1BHC_H 1BHC_I 1BHC_J 1BPI_A 1BTH_P 1BTH_Q 1BZ5_A 1BZ5_B 1BZ5_C 1BZ5_D 1BZ5_E 1BZX_I 1CBW_D 1CBW_I 1D0D_B 1EAW_B 1EAW_D 1FY8_I 1MTN_D 1MTN_H 1TPA_I 2FTL_I 2HEX_A 2HEX_B 2HEX_C 2HEX_D 2HEX_E 2IJO_I 2KAI_I 2PTC_I 2R9P_E 2R9P_F 2R9P_G 2R9P_I 2RA3_C 2RA3_I 2TGP_I 2TPI_I 3BTK_I 3FP6_I 3FP8_I 3GYM_I 3GYM_J 3LDI_A 3LDI_B 3LDI_C 3LDI_D 3LDI_E 3LDJ_A 3LDJ_B 3LDJ_C 3LDM_A 3LDM_B 3LDM_C 3LDM_D 3LDM_E 3TPI_I 3U1J_E 4DG4_C 4DG4_E 4DG4_F 4DG4_H 4PTI_A 4WWY_C 4WWY_I 4Y0Y_I 5PTI_A 6F1F_A 6F1F_B 6F1F_C 6F1F_D 6F1F_E 6PTI_A 9PTI_A 1YKT_B 4WXV_C 4WXV_I 5YVU_I 
1AAP_A 1AAP_B 1TAW_B 1ZJD_B 1BRC_I 1CA0_D 1CA0_I 3L33_E 3L33_F 3L33_G 3L33_H 
3BYB_A 3BYB_B 3BYB_C 3UIR_C 3UIR_D 5ZJ3_A 5ZJ3_B 5ZJ3_C 3D65_I 
3WNY_A 3WNY_B 3WNY_C 3WNY_E 3WNY_F 3WNY_G 3WNY_H 3WNY_I 
1Y62_A 1Y62_B 1Y62_C 

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	run_blastclust
	1

rule run_blastclust:
    input: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta
    output: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz
    jobid: 0
    wildcards: db=intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary, S=99, L=0.95

blastclust -i intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.fasta -L 0.95 -S 99 | gzip > intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz
Finished job 0.
1 of 1 steps (100%) done


Finally, filter the intersection table to remove redundancy given the clustering; the representative sequence for each cluster is selected according to the best resolution of the structure

In [7]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt
head chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt 

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
5PTI_A	1.00	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58	1	16.18	11.93	0.000
1AAP_A	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58	0.8785	8.378	8.472	0.944
3BYB_A	1.63	Pseudonaja_textilis	KDRPDFCELPADTGPCRVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCAA	59	0.8603	10.23	9.402	0.867
3WNY_A	1.30	Bos_taurus	RPAFCLEPPYAGPGKARIIRYFYNAKAGAAQAFVYGGVRAKRNNFASAADALAACAAAGGKKK	63	0.934	10.78	9.662	0.551
1Y62_A	2.45	Conus_striatus	KDRPSLCDLPADSGSGTKAEKRIYYNSARKQCLRFDYTGQGGNENNFRRTYDCQRTCLYT	60	0.8862	10.53	9.544	0.897
1F7Z_I	1.55	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGAIGPWENL	65	0.9549	11.71	10.09	0.317
4U30_W	2.50	Homo_sapiens	TVAACANLPIVRGPCRAFIQLWAFDAVKGKCVLFPYGGCQGNGNKFYSEKECREYCGVP	59	0.7769	7.288	7.874	1.001
4ISO_B	2.01	Homo_sapiens	QTEDYCLASNKVGRCRGSFPRWYYDPTEQICKSFVYGGCLGNKNNYLREEECILACRGVQ	60	0.8457	6.038	7.132

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	extract_representative_sequences
	1

rule extract_representative_sequences:
    input: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz, intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt
    jobid: 0
    wildcards: sequences=intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary, S=99, L=0.95

for cluster in $(zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz | sed '1d' | sed 's/ /%/g'); do zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary.txt.gz | grep -w -f <(echo $cluster | sed 's/%/\n/g') | sort -n -k2 | sed -n '1p'; done > chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt
Finish

Prepare the selected sequences in fasta format for further processing with hmmer

In [8]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.fasta.gz
zcat chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.fasta.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
>5PTI_A
RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA
>1AAP_A
VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA
>3BYB_A
KDRPDFCELPADTGPCRVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCAA
>3WNY_A
RPAFCLEPPYAGPGKARIIRYFYNAKAGAAQAFVYGGVRAKRNNFASAADALAACAAAGGKKK
>1Y62_A
KDRPSLCDLPADSGSGTKAEKRIYYNSARKQCLRFDYTGQGGNENNFRRTYDCQRTCLYT


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	filtered_intersection2fasta
	1

rule filtered_intersection2fasta:
    input: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.fasta.gz
    jobid: 0
    wildcards: sequences=intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary, S=99, L=0.95

cat chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt | awk '{print ">"$1"\n"$4}' | gzip > chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.fasta.gz
Finished job 0.
1 of 1 steps (100%) done
